# Updating facts (on the fly)

This notebook is going to show the same process than the `updating_documents` notebook, but with facts instead. Nothing fancy, just to show you how it works.

In [ ]:
import hybridagi.core.graph_program as gp

main = gp.GraphProgram(
    name = "main",
    description = "The main program",
)

main.add(gp.Action(
    id = "search_facts",
    purpose = "Search for relevant facts to answer the Objective's question",
    tool = "FactSearch",
    prompt = "Please use the Objective's question to infer the search query",
))

main.add(gp.Decision(
    id = "check_context",
    purpose = "Check if the answer to the Objective's question is in your Context",
    question = "Is the answer to the Objective's question in your context?"
))

main.add(gp.Action(
    id = "answer_context_based",
    purpose = "Answer to the Objective's question based on your Context",
    tool = "Speak",
    prompt = "Answer to the Objective's question, if there is relevant information in your Context, please use it",
))

main.add(gp.Action(
    id = "answer",
    purpose = "Answer to the Objective's question",
    tool = "Speak",
    prompt = "Answer to the Objective's question, don't say it is based on your search just answer",
))

main.add(gp.Action(
    id = "save_answer",
    purpose = "Save the answer to the Objective Question into memory",
    tool = "AddFact",
    prompt = "Use the answer in your context to infer the document to save, never explain what you are doing",
))

main.connect("start", "search_facts")
main.connect("search_facts", "check_context")
main.connect("check_context", "answer_context_based", label="Yes")
main.connect("check_context", "answer", label="No")
main.connect("answer", "save_answer")
main.connect("answer_context_based", "end")
main.connect("save_answer", "end")

main.build()

In [ ]:
# We add the programs into memory
from hybridagi.memory.integration.falkordb import FalkorDBProgramMemory, FalkorDBFactMemory
from hybridagi.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(
    model_name_or_path = "all-MiniLM-L6-v2",
    dim = 384,
)

program_memory = FalkorDBProgramMemory(
    index_name="update_facts",
    embeddings = embeddings,
    wipe_on_start=True,
)

program_memory.update(main)

# Then we instantiate the document memory for later use

fact_memory = FalkorDBFactMemory(
    index_name="update_facts",
    embeddings = embeddings,
    wipe_on_start=True,
)

In [3]:
# Now we setup the agent and tools
import dspy
from hybridagi.core.datatypes import AgentState
from hybridagi.core.pipeline import Pipeline
from hybridagi.embeddings import SentenceTransformerEmbeddings
from hybridagi.modules.agents import GraphInterpreterAgent
from hybridagi.modules.splitters import DocumentSentenceSplitter
from hybridagi.modules.embedders import FactEmbedder
from hybridagi.modules.extractors import LLMFactExtractor
from hybridagi.modules.deduplicators import EntityDeduplicator
from hybridagi.modules.retrievers.integration.local import FAISSFactRetriever 
from hybridagi.modules.agents.tools import (
    SpeakTool,
    FactSearchTool,
    AddFactTool,
)

embeddings = SentenceTransformerEmbeddings(
    model_name_or_path = "all-MiniLM-L6-v2",
    dim = 384,
)

pipeline = Pipeline()

# The facts processing pipeline, takes an input document and convert it to facts

pipeline.add("chunk_docs", DocumentSentenceSplitter(
    method = "word",
    chunk_size = 100,
    chunk_overlap = 0,
    separator = " ",
))
pipeline.add("extract_facts", LLMFactExtractor())
pipeline.add("deduplicate_entities", EntityDeduplicator(method="exact"))
pipeline.add("embed_facts", FactEmbedder(embeddings=embeddings))

agent_state = AgentState()

tools = [
    SpeakTool(
        agent_state = agent_state,
    ),
    FactSearchTool(
        retriever = FAISSFactRetriever(
            fact_memory = fact_memory,
            embeddings = embeddings,
            distance = "cosine",
            max_distance = 1.0,
            k = 5,
            reranker = None,
        ),
    ),
    AddFactTool(
        fact_memory = fact_memory,
        pipeline = pipeline, # Here we bind the document processing pipeline to our tool
    ),
]

lm = dspy.OllamaLocal(model='mistral', max_tokens=1024, stop=["\n\n\n"])
dspy.configure(lm=lm)

agent = GraphInterpreterAgent(
    program_memory = program_memory,
    agent_state = agent_state,
    tools = tools,
)

In [ ]:
from hybridagi.core.datatypes import Query

# Now we can test our system
result = agent(Query(query="What the definition of a neuro symbolic AI"))

print(result.final_answer)

In [ ]:
from hybridagi.core.datatypes import Query

# Now we can test our system
result = agent(Query(query="What the definition of a neuro symbolic AI"))

print(result.final_answer)